# Calculating state-by-state implied infection numbers

This notebook tries to compute what the full infection numbers in the past and present likely were/are.

It does so in the past by blending variables for "median days from infection to death" and "infection fatility rate" (IFR) with smoothed death rates. In other words, days_to_death days before date D, there must have been roughly (deaths_on_date_D / IFR) infections to end up with a given number of deaths on date D.

It does in the present to looking at what percentage of infections were confirmed on the last day calculated in the past, and applying that percentage to the new infections found since then. That doesn't quite take into account if there is a significant ramping of testing during that time, but it should be close enough.

The principal source of death data is files from the NY Times, supplemented by a more accurate DateOfDeath.csv from Massachusetts. The source of testing data is The COVID Tracking Project, maintained by The Atlantic.

NOTE: Prior to running this notebook, you should retrieve the latest DateOfDeath.csv file by:

1. going to https://www.mass.gov/info-details/covid-19-response-reporting,
2. downloading the raw data zip from the line saying "Raw data used to create the dashboard is available here:"
3. copying the DateofDeath.csv in that file to the same directory as the notebook

Yeah, that could be automated. Just haven't done it yet...

In [ ]:
%matplotlib inline
import numpy
import pandas
import matplotlib
import matplotlib.pyplot as plt

from common import load_data, smooth_series, calc_mid_weekly_average

In [ ]:
# Earliest date that there is sufficient data for all states, including MA
EARLIEST_DATE = pandas.Period('2020-03-10', freq='D')
LATEST_DATE = pandas.Period('2020-08-07', freq='D')
LATEST_DATE = None


In [ ]:
meta, nyt_stats, ct_stats = load_data(EARLIEST_DATE, LATEST_DATE)

In [ ]:
ct_stats.tail()

### Put the two datasets together

In [ ]:
ct1 = ct_stats.set_index(['ST', 'Date']).sort_index()[['Pos', 'Tests']]
nyt1 = nyt_stats.set_index(['ST', 'Date']).sort_index()[['Deaths']]
stats = ct1.join(nyt1)
stats.head()

### Calculate new stats, state by state

In [ ]:
def calc_state_stats(state, state_stats, meta):
    st = state_stats.groupby('Date').sum().sort_index().copy()

    st['ST'] = state
    st['Pop'] = meta.loc[state].Pop

    # Smooth series that might not be reported daily in some states
    st.Pos = smooth_series(st.Pos)
    st.Deaths = smooth_series(st.Deaths)

    # Prep for 7-day smoothing calculations
    st7 = st.shift(7)
    
#     # Calculate daily confirmed positive cases and deaths
#     st['Confirms'] = (st.Pos - st.shift().Pos)
#     st['Daily'] = (st.Deaths - st.shift().Deaths)

#     # Calculate both as trailing 7-day averages
#     st['Confirms7'] = (st.Pos - st7.Pos) / 7
#     st['Deaths7'] = (st.Deaths - st7.Deaths) / 7

#     # Convert into central 7-day mean, with special handling for last three days
#     specs = (
#         (8., numpy.array([1., 1., 1.1, 1.2, 1.4, 1.2, 1.1])),
#         (9., numpy.array([1., 1.1, 1.1, 1.2, 1.4, 1.7, 1.5])),
#         (10., numpy.array([1., 1.1, 1.2, 1.3, 1.4, 1.8, 2.2])),
#     )
#     for col in [-2, -1]:
#         st.iloc[:, col] = st.iloc[:, col].shift(-3)
#         dailies = st.iloc[-7:, col-2].values
#         vals = [((dailies * factors).sum() / divisor) for divisor, factors in specs]
#         st.iloc[-3:, col] = vals

    st['Confirms'], st['Confirms7'] = calc_mid_weekly_average(st.Pos)
    st['Daily'], st['Deaths7'] = calc_mid_weekly_average(st.Deaths)

    return st.reset_index().set_index(['ST', 'Date']).copy()

In [ ]:
meta_tmp = meta.set_index('ST')[['Pop']]

In [ ]:
states = [calc_state_stats(state, df, meta_tmp)
          for state, df in stats.reset_index().groupby('ST')]
states[-17].tail()

In [ ]:
def get_infections_df(states, death_lag, ifr_high, ifr_low, incubation, infectious,
                      max_confirmed_ratio=0.7):
    new_states = []
    for state in states:
        state = state.copy()

        # Calculate the IFR to apply for each day
        ifr = pandas.Series(numpy.linspace(ifr_high, ifr_low, len(state)), index=state.index)
        # Calculate the infections in the past
        infections = state.shift(-death_lag).Deaths7 / ifr
        
        # Calculate the min infections based on max_confirmed_ratio
        min_infections = state.Confirms7 / max_confirmed_ratio
        infections = infections.combine(min_infections, max, 0)

        # Find out the ratio of infections that were detected on the last date in the past
        last_date = infections.index[-(death_lag+1)]
        last_ratio = infections.loc[last_date] / (state.loc[last_date, 'Confirms7'] + 1)

        # Apply that ratio to the dates since that date
        infections.iloc[-death_lag:] = state.Confirms7.iloc[-death_lag:] * last_ratio

        state['NewInf'] = infections
        state['TotInf'] = infections.cumsum()
        state['ActInf'] = infections.rolling(infectious).sum().shift(incubation)
        state['ActKnown'] = state.Confirms7.rolling(infectious).sum()
        state['ActUnk'] = state.ActInf - state.ActKnown
        state['AIPer1000'] = state.ActInf / state.Pop / 1000.
        state['AUPer1000'] = state.ActUnk / state.Pop / 1000.
        state['PctFound'] = state.Confirms7 / (state.NewInf + 1)
        new_states.append(state)

    return pandas.concat(new_states)

In [ ]:
infected_states = get_infections_df(states, 19, 0.011, 0.004, 4, 10)
print(infected_states.NewInf.sum())
fam = infected_states.reset_index()[['Date', 'NewInf']].groupby('Date').sum().plot(
    title="Infection Estimations, 19 median days to death, IFR improving 1.1% - 0.4%", figsize=(10,5),
    legend=None, ylim=0
)

In [ ]:
fam = infected_states.reset_index()[['Date', 'Deaths7']].groupby('Date').sum().plot(
    title="Deaths", figsize=(10,5),
    legend=None, ylim=0
)

In [ ]:
foozle = infected_states.reset_index()[['Date', 'NewInf', 'Deaths7']].groupby('Date').sum()
foozle.columns = ['Infections', 'Deaths']
ax = foozle.plot(
    title="Daily Infections vs. Deaths, 19 median days to death, IFR improving 1.1% - 0.4%",
    secondary_y='Deaths', figsize=(10,5), ylim=0)
#ax2 = ax.twinx()

In [ ]:
foo = infected_states.reset_index().set_index(['Date', 'ST']).sort_index()
foo = foo[['Pop', 'Confirms7', 'Deaths7', 'NewInf', 'AIPer1000', 'AUPer1000', 'PctFound']]


In [ ]:
faz = foo.loc['2020-08-08', :].sort_values('AUPer1000', ascending=False).copy()
faz = faz.reset_index()[['ST', 'Pop', 'Confirms7', 'Deaths7', 'AUPer1000', 'PctFound']]
faz.columns = ['ST', 'Pop', 'Cases', 'Deaths', 'ActUnk1000', 'PctFound']
faz = faz.set_index('ST')
faz.head(15)

In [ ]:
faz.tail(10)

In [ ]:
faz[faz.PctFound <= 0.3].sort_values('PctFound')

In [ ]:
fam = foo.reset_index()
fam = fam[fam.ST == 'NJ']
fam.set_index('Date').tail(20)

In [ ]:
testing = ct_stats[['ST', 'Date', 'Pos', 'Neg']].copy()
testing['Total'] = testing.Pos + testing.Neg
testing['Daily'], testing['Smoothed'] = calc_mid_weekly_average(testing.Total)
testing.tail()

In [ ]:
SCENARIOS = (('1.0% - 0.6%', 20, 0.01, 0.006), )

In [ ]:
raise ValueError()

## Now for the charts...

In [ ]:
SCENARIOS = (('20', 20, 0.01, 0.01), ('18', 18, 0.01, 0.01), ('16', 16, 0.01, 0.01), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="New Infections Estimates, varying average days to death, IFR = 1.0%", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.3%', 18, 0.013, 0.013), ('1.0%', 18, 0.01, 0.01), ('0.7%', 18, 0.007, 0.007), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="New Infections Estimates, varying IFR, days to death = 18", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.2% - 0.8%', 18, 0.012, 0.008), ('1.0% - 0.7%', 18, 0.01, 0.007), ('0.9% - 0.6%', 18, 0.009, 0.006), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Infection Estimations, improving IFR, days to death = 18", figsize=(10,5))

In [ ]:
SCENARIOS = (('1.0% - 0.6%', 20, 0.01, 0.006), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Infection Estimations, my hunch, 20 median days to death", figsize=(10,5))

In [ ]:
df.sum()

In [ ]:
df.tail()

In [ ]:
SCENARIOS = (('1.2% - 0.5%', 21, 0.012, 0.005), )

df = get_infections_df(SCENARIOS)
foo = df.plot(title="Worst case? 21 days to death, improving IFR", figsize=(10,5))

In [ ]:
numpy.linspace(1, 2, 3)

In [ ]:
tests = ct_stats[['ST', 'Date', 'Pos', 'Neg']].copy()
tests['Tests'] = tests.Pos + tests.Neg
tests['Daily'] = tests.Tests - tests.shift().Tests
tests = tests[['ST', 'Date', 'Daily']]
last_5 = tests[tests.Date >= '2020-07-30'].groupby('ST').sum()
prior_5 = tests[(tests.Date > '2020-07-24') & (tests.Date < '2020-07-30')].groupby('ST').sum()
diffs = ((prior_5 - last_5) / 5)
diffs.sort_values('Daily', ascending=False)

In [ ]:
last_5 = ct_stats[ct_stats.Date >= '2020-07-30']
last_5.tail(10)

In [ ]:
infected_states.reset_index()[['Date', 'Deaths7']].groupby('Date').sum().tail(30)

In [ ]:
infected_states.tail()

In [ ]:
# ['AL', 'AZ', 'FL', 'GA', 'LA', 'MS', 'NV', 'SC', 'TX']
infected_states.loc['AL', :].NewInf.plot()
infected_states.loc['TX', :].NewInf.plot()

In [ ]:
states = ['AL', 'AZ', 'FL', 'GA', 'LA', 'MS', 'NV', 'SC', 'TX']
states = list(infected_states.index.get_level_values(0).unique())
fig, axes = plt.subplots(len(states), figsize=(10, 4*len(states)))
for i, state in enumerate(states):
    try:
        infected_states.loc[state, :].AIPer1000.plot(ax=axes[i], title=state)
    except:
        pass


In [ ]:
infected_states.index.get_level_values(0).unique()

In [ ]:
s = pandas.Series([10, 15, 25, 26, 39, 46, 57, 61, 87, 100])
d, m7 = calc_mid_weekly_average(s)
d, m7

In [ ]:
d.sum(), m7.sum()